In [ ]:
%load_ext autoreload
%autoreload 2
%env CUBLAS_WORKSPACE_CONFIG=:4096:8
from GraPL import hyperparameter_profiles, segment_bsds
# automatically reload modules when they have changed
import glob
import tqdm
import matplotlib.pyplot as plt
from GraPL.evaluate import bsds_score_directory
import numpy as np
import json
import os

In [ ]:
def run_experiment(base_params, initialization_methods, num_trials=2):
    results_base_dir = f'experiment_results/initialization'
    aggregate_scores = {}
    for initialization_method in initialization_methods:
        print(f'Running experiment with initialization_method={initialization_method}...')
        results_dir = f'{results_base_dir}/{initialization_method}'
        trials = []
        mean_over_trials = {}
        with tqdm.tqdm(total=num_trials * 200) as progress_bar:
            for trial_num in range(num_trials):
                params = base_params.copy()
                params['initialization_method'] = initialization_method
                params['seed'] = trial_num
                trial_results_dir = f'{results_dir}/{trial_num}'
                trial_scores = segment_bsds(results_dir=trial_results_dir, progress_bar=progress_bar, **params)
                image_ids = list(trial_scores.keys())
                metrics = trial_scores[image_ids[0]].keys()
                trial_scores = {metric: np.mean([trial_scores[id][metric] for id in image_ids]) for metric in metrics}
                trials.append(trial_scores)
        for metric in trials[0]:
            mean_over_trials[metric] = np.mean([trial[metric] for trial in trials])
        aggregate_scores[initialization_method] = mean_over_trials
        print(f'Scores for initialization_method={initialization_method}: {mean_over_trials["accuracy"]}')
    return aggregate_scores

base_params = hyperparameter_profiles["best_miou"]

initialization_methods = ["random", "knn", "kmeans", "slic"]

aggregate_scores = run_experiment(base_params, initialization_methods, num_trials=10)